In [1]:
import os
import time
import google.generativeai as genai
from tqdm.notebook import tqdm # Para la barra de progreso VERSION JUPYTER

C:\Users\Usuario\AppData\Local\Temp\ipykernel_19236\92653507.py:3: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [2]:
# --- CONFIGURACIÓN ---
# Reemplaza con tu API Key real
with open('gemini.txt', 'r') as f:
    clave = f.read()
API_KEY = clave

empresa = "TREX"

CARPETA_SQL = "C://Users//Usuario//OneDrive//MiOneDrive//Chamba//Marita//SPs//"+empresa  # La ruta donde están tus 500 archivos
MODELO = "gemini-2.5-flash" # Modelo rápido y económico
carpeta_output = "C://MiJupyterNotebook//output_docu_sps//"

# Configura la API
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel(MODELO)

In [ ]:
len([f for f in os.listdir(CARPETA_SQL) if f.endswith('.sql')])

In [3]:
# Obtener lista de archivos .sql
archivos = [f for f in os.listdir(CARPETA_SQL) if f.endswith('.sql')]
archivos = archivos[200:]
archivos
 # 'dbo.CalcularSubTotalesEERRAcumulados_Soles_PPTO.StoredProcedure.sql']

['dbo.sp_CompararTotalCuentasPorCobrarComerciales_Validacion02_Nota03_A.StoredProcedure.sql',
 'dbo.sp_CompararTotalCuentasPorCobrarDiversas_Validacion04_Nota03_A.StoredProcedure.sql',
 'dbo.sp_CompararTotalCuentasPorPagarComerciales_Validacion06_Nota03_A.StoredProcedure.sql',
 'dbo.sp_CompararTotalDeterioradoConEstimacionCuentasCobranzaDudosa_Validacion09_Nota02_C.StoredProcedure.sql',
 'dbo.sp_CompararTotalDeteriorado_Validacion07_Nota02_C.StoredProcedure.sql',
 'dbo.sp_CompararTotales_Validacion21_Nota11.StoredProcedure.sql',
 'dbo.sp_CompararTotales_Validacion22_Nota11.StoredProcedure.sql',
 'dbo.sp_CompararTotalExistencias_Validacion04_ME.StoredProcedure.sql',
 'dbo.sp_CompararTotalIngresosOperativos_Validacion01_EERRAcumulado.StoredProcedure.sql',
 'dbo.sp_CompararTotalIngresosOperativos_Validacion07_EERRMensual.StoredProcedure.sql',
 'dbo.sp_CompararTotalNoDeteriorado_Validacion06_Nota02_C.StoredProcedure.sql',
 'dbo.sp_CompararTotalNota5A_Validacion02_Nota5_C.StoredProcedure.sq

In [4]:
len(archivos)

345

In [7]:
procesar_archivos(archivos)

Encontrados 345 archivos. Iniciando...


  0%|          | 0/345 [00:00<?, ?it/s]

In [5]:
def generar_documentacion(codigo_sql,nombre_archivo):
    """
    Envía el código SQL al LLM y retorna el bloque de comentarios.
    """
    
    prompt = f"""
    Rol: Eres un DBA Senior experto en T-SQL y procesos de consolidación financiera.
    Tarea: Generar documentación técnica estandarizada para el siguiente Store Procedure.
    
    Instrucciones de Formato:
    1. NO incluyas saludos, introducciones ni texto de relleno ("Aquí está tu análisis...").
    2. NO uses bloques de código (```markdown, ```text, etc). Devuelve texto plano directo.
    3. Sigue estrictamente la plantilla proporcionada abajo.
    4. Si un campo no aplica, escribe "N/A".
    
    Instrucciones de Análisis:
    - Categoria: Elige SOLAMENTE UNA de esta lista: [Consolidación de montos, Validación/Comparación de montos, Conversión local a dólar, Conversión dólar a sol, Otros].
    - Descripción: Enfócate en la lógica de negocio (el "por qué" y "para qué"), no describas línea por línea el código SQL.
    - Parámetros: Indica el nombre, tipo de dato y explica brevemente su uso. Si detectas que no se usa en el cuerpo del script, márcalo como "(NO UTILIZADO)".
    - Tablas: Separa claramente entre tablas que solo se leen (Source) y tablas que son modificadas (Target: INSERT, UPDATE, DELETE, MERGE).
    - Store Procedures ejecutados: Indica si, durante el proceso, se ejecuta otro/s store procedure/s.
    
    --- PLANTILLA DE INICIO ---
    Nombre Archivo: {nombre_archivo}
    Categoría: [CATEGORIA_AQUI]
    ----------------------------------------
    Descripción del Proceso:
    [RESUMEN_DE_NEGOCIO_AQUI]
    ----------------------------------------
    Parámetros:
    - @Parametro1 (Tipo): Explicación...
    - @Parametro2 (Tipo): Explicación... (NO UTILIZADO)
    ----------------------------------------
    Tablas Afectadas:
    [LECTURA]: TablaA, TablaB
    [ESCRITURA]: TablaC
    ----------------------------------------
    Store Procedures adicionales:
    - SP #1...
    - SP #2...
    --- PLANTILLA DE FIN ---
    
    Código SQL a analizar:
    {codigo_sql}
    """
    
    try:
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        return f"/* ERROR GENERANDO DOCUMENTACION: {str(e)} */"

In [6]:
def procesar_archivos(l_archivos):
    print(f"Encontrados {len(l_archivos)} archivos. Iniciando...")

    # Usamos tqdm para ver una barra de progreso
    for nombre_archivo in tqdm(l_archivos):
        ruta_completa = os.path.join(CARPETA_SQL, nombre_archivo)
        
        # with open(ruta_completa, 'r', encoding='utf-8') as f:
        with open(ruta_completa, 'r', encoding='utf-16') as f:            
            codigo_sp = f.read()

        # Generar docs
        doc_header = generar_documentacion(codigo_sp,nombre_archivo)

        #ruta archivo output
        ruta_new_archivo = carpeta_output+"DOCU__"+nombre_archivo+".txt"
        
        with open(ruta_new_archivo, "w", encoding="utf-8") as new_archivo:
            new_archivo.write(doc_header)
        
        # IMPORTANTE: Pausa para evitar errores de Rate Limit (Límite de peticiones)
        time.sleep(5)

VER ERRORES

In [ ]:
errores = []

for nombre_archivo in tqdm(archivos):
    ruta_completa = os.path.join(CARPETA_SQL, nombre_archivo)

    try:
        with open(ruta_completa, 'r', encoding='utf-16') as f:
            a = f.read()
    except Exception as e:
        errores.append(nombre_archivo)

print("Números de archivos errados: "+str(len(errores)))
print(errores)

In [ ]:
with open(os.path.join(CARPETA_SQL, 'dbo.AjustesDeSaldos.StoredProcedure.sql'), 'r', encoding='utf-16') as f:
            a = f.read()
print(a)

In [ ]:
    # prompt = f"""
    # Actúa como un DBA experto. Analiza el siguiente Store Procedure de SQL Server.
    # Este pertence al proceso de consolidación financiera de una corporación con multiples empresas y que aplica varias reglas de negocio.
    # Genera un bloque de documentación estándar el cual pueda escribirse y leerse en un archivo .txt
    
    # El formato debe incluir estrictamente:
    # - Nombre: el nombre del archivo
    # - Categoria: Agregalo en una de las siguientes categorias:
    #     >Consolidación de montos
    #     >Validación/Comparación de montos
    #     >Conversión local a dólar
    #     >Conversión dólar a sol
    #     >Otros
    # - Descripción: Un resumen de entre 5 a 10 líneas sobre qué proceso de negocio realiza.
    # - Parametros: Lista de parámetros y su uso probable. Menciona también si es una variable que nunca se usa en el store procedure.
    # - Tablas afectas: Tablas principales que se leen (SELECT) o modifican (INSERT/UPDATE/DELETE).
        
    # Código SQL a analizar:
    # {codigo_sql}
    # """

In [ ]:
# a = model.generate_content(f"""
# Resume en dos parrafos la guerra del pacifico
# """)
# print(a.text)
# with open("resumen_guerra.txt", "w", encoding="utf-8") as archivo:
#     archivo.write(a.text)